In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd


In [ ]:
import numpy as np



In [ ]:
import matplotlib.pyplot as pyplot
%matplotlib inline 

In [ ]:
facial_raw_train = pd.read_csv("sentiment_data.csv")
face_lst=[]
for i in range (len(facial_raw_train)):
    face_lst.append(facial_raw_train["feature"][i].split())
images=np.asarray(face_lst,dtype=np.float32)
img_lst=[]
for j in range (len(images)):
    img_lst.append(images[j].reshape(48,48))
X_total=np.asarray(img_lst, dtype=np.float32)
X_total=np.reshape(X_total, (len(X_total), 48, 48, 1))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rotation_range=30)
train_datagen.fit(X_total)

In [ ]:
Y_lst=[]
for i in range(len(facial_raw_train["label"])):
    Y_lst.append(facial_raw_train["label"][i])
    
Y_total=np.asarray(Y_lst,dtype=np.float32)

In [ ]:
#data augmentation. some typical sample images from the processed dataset.
for i in range(0, 9):
    pyplot.subplot(330 + 1 + i)
    pyplot.imshow(X_total[0:9][i].reshape(48, 48), cmap=pyplot.get_cmap('gray'))
# show the plot
pyplot.show()

Y_total[0:9]

In [ ]:
##data augmentation. some typical sample images from the raw dataset.



from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range=30)
datagen.fit(X_total[0:9])
for X_batch, y_batch in datagen.flow(X_total[0:9],Y_total[0:9], batch_size=9):
	# create a grid of 3x3 images
	for i in range(0, 9):
		pyplot.subplot(330 + 1 + i)
		pyplot.imshow(X_batch[i].reshape(48, 48), cmap=pyplot.get_cmap('gray'))
	# show the plot
	pyplot.show()
	break
print(X_batch.shape)
y_batch

In [ ]:
def split_train_test(X_total,Y_total, test_ratio):
    if (len(X_total)!=len(Y_total)):
        return false
    shuffled_indices = np.random.permutation(len(X_total))
    test_set_size = int(len(X_total) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return X_total[train_indices],Y_total[train_indices], X_total[test_indices],Y_total[test_indices]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
import cv2

def pre_process(fileName,test_ratio):
    """
    The function load provided CSV dataset and further reshape, rescale the data for feeding
    """
    facial_raw_train = pd.read_csv(fileName)
    face_lst=[]
    for i in range (len(facial_raw_train)):
        face_lst.append(facial_raw_train["feature"][i].split())
    images=np.asarray(face_lst,dtype=np.float32)
    img_lst=[]
    
    
    #Resizing the image (48 * 48) to (32 * 32) for decresing paramemeter when training models
    for j in range (len(images)):
        img=images[j].reshape(48,48)
        img=cv2.resize(img,(32,32))
        img_lst.append(img)
        
    X_total=np.asarray(img_lst, dtype=np.float32)
    X_total=np.reshape(X_total, (len(X_total), 32, 32, 1))
    
    Y_lst=[]
    for i in range(len(facial_raw_train["label"])):
        Y_lst.append(facial_raw_train["label"][i])
    
    Y_total=np.asarray(Y_lst,dtype=np.float32)
    Y_total = to_categorical (Y_total)
    
    
    
    
    print (X_total.shape)
    
    X_train,Y_train,X_test,Y_test = split_train_test(X_total,Y_total, test_ratio) 
    
    
   
    train_datagen = ImageDataGenerator(rotation_range=30)
    train_datagen.fit(X_train)
    
    test_datagen = ImageDataGenerator(rotation_range=30)
    test_datagen.fit(X_test)
    
    
    import pickle
    fileObject=open("processed_data",'wb')
    pickle.dump((X_train,Y_train, X_test, Y_test, train_datagen, test_datagen), fileObject)
    

In [ ]:
pre_process("sentiment_data.csv",0.3)




In [ ]:
import pickle
fileObject=open("processed_data",'rb')
X_train,Y_train,X_test,Y_test, train_datagen, test_datagen = pickle.load(fileObject)

In [ ]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization

import numpy as np

In [ ]:
from __future__ import division, print_function, absolute_import

In [ ]:

input_shape=(32,32,1)
model= Sequential()

model.add(Conv2D(32, kernel_size=(3,3),activation='relu',input_shape = input_shape, padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D((2,2)))
           
           
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(7,activation = 'softmax'))
           
model.summary()
           

In [ ]:
from keras.models import model_from_json

In [ ]:
from keras.optimizers import Adam
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_datagen.flow(X_train, Y_train, batch_size=256),
                    steps_per_epoch=len(X_train) // 256,
                    epochs=30,
                    verbose=1)

In [ ]:
model.evaluate (X_test,Y_test)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
from keras.models import Model, load_model
from keras.applications.vgg16 import VGG16

In [ ]:
vgg16_model = VGG16(weights = 'imagenet', include_top = False)
x = vgg16_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(7, activation = 'softmax')(x)
vgg_model = Model(input = vgg16_model.input, output = predictions)

In [ ]:
# freeze all layers of the pre-trained model
for layer in vgg16_model.layers:
    layer.trainable = False
vgg_model.summary()

In [ ]:
import cv2
V_X_train = [cv2.cvtColor(cv2.resize(i, (32, 32)), cv2.COLOR_GRAY2RGB) for i in X_train]
V_X_test=[cv2.cvtColor(cv2.resize(i, (32, 32)), cv2.COLOR_GRAY2RGB)
           for i in X_test]
V_X_train = np.concatenate([arr[np.newaxis] for arr in V_X_train]).astype('float32')
V_X_test = np.concatenate([arr[np.newaxis] for arr in V_X_test]).astype('float32')

In [ ]:
from keras.optimizers import Adam
vgg_model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vgg_model.fit_generator(train_datagen.flow(V_X_train, Y_train, batch_size=256),
                    steps_per_epoch=len(V_X_train) // 256,
                    epochs=20,
                    verbose=1)

In [ ]:
vgg_model.evaluate (V_X_test,Y_test)